In [1]:
%%capture

import ROOT
import os
from dask.distributed import Client, PipInstall, WorkerPlugin
import json
from samplesUL import *
import sys 
from dask_jobqueue import HTCondorCluster
from distributed.diagnostics.plugin import UploadFile

distributed = True
MT = False
#redirector = "root://webdav-7-3-25.recas.ba.infn.it:8080/" # Bari 1 - read issues
#redirector = "root://xr-4-4-9-3.recas.ba.infn.it:8080/" # Bari 2 - OK
#redirector = "root://t2-xrdcms.lnl.infn.it:7070/" # Legnaro - OK
redirector = "file:///shared-scratch/cms/" # Local FS shared ssd
#redirector = "file:///scratch/cms/" # Local storage nvme
maxNfilespersample = 99999 #5 lower this number just for debugging purposes: 99999 prod.
sched_port = 26604 #used only in distributed mode
nPartitions = 92*3  #used only in distributed mode (golden rule 3*Nworkers)

os.environ['X509_USER_PROXY'] = '/opt/workspace/persistent-storage/proxy'
os.environ['X509_CERT_DIR'] = '/cvmfs/cms.cern.ch/grid/etc/grid-security/certificates/'

if distributed != True and MT == True:
    ROOT.ROOT.EnableImplicitMT()

def print_proc_diskstats():
    import subprocess
    result = subprocess.run(["cat", "/proc/diskstats"], capture_output=True, text=True)
    return result.stdout
    
def dir_size():
    import subprocess
    folder_path = "/scratch"
    result = subprocess.run(["du", "-sh", folder_path], capture_output=True, text=True)
    return result.stdout.strip()

def get_memory_info():
    import subprocess
    result = subprocess.run(["free", "-h"], capture_output=True, text=True)
    return result.stdout

def set_proxy(dask_worker):
    import os
    import shutil
    working_dir = dask_worker.local_directory
    print(working_dir)
    os.environ['X509_USER_PROXY'] = working_dir + '/proxy'
    os.environ['X509_CERT_DIR']="/cvmfs/grid.cern.ch/etc/grid-security/certificates/"
    os.environ['EXTRA_CLING_ARGS'] = "-O2"
    os.popen('rm -f "./*.csv"')
    try:
        shutil.copyfile(working_dir + '/proxy', working_dir + '/../../../proxy')
    except:
        pass
    try:
        os.chmod(working_dir + '/proxy', 0o400)
    except:
        pass
    try:
        os.chmod(working_dir + '/../../../proxy', 0o400)
    except:
        pass
        
    return os.environ.get("X509_USER_PROXY")

In [2]:
'''
%%bash

# xrdcp where I am and then cp from terminal because the notebook is a singularity container without root permissions
xrdcp "root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/ZGToLLG_01J_5f_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/AF4D5462-30E8-EA43-AA8A-A1EADA9796BC.root" "./AF4D5462-30E8-EA43-AA8A-A1EADA9796BC.root"
xrdcp "root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/WZ_TuneCP5_13TeV-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/120000/936C5557-15D6-B74C-9260-582B71C8B1C3.root" "./936C5557-15D6-B74C-9260-582B71C8B1C3.root"
xrdcp "root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv2/GluGluToWWToTNMN_TuneCP5_13TeV_MCFM701_pythia8/NANOAODSIM/106X_mc2017_realistic_v8-v1/270000/ABA8AAB6-89A8-2A4D-B574-82CB4162656E.root" "./ABA8AAB6-89A8-2A4D-B574-82CB4162656E.root"
'''

'\n%%bash\n\n# xrdcp where I am and then cp from terminal because the notebook is a singularity container without root permissions\nxrdcp "root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/ZGToLLG_01J_5f_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/AF4D5462-30E8-EA43-AA8A-A1EADA9796BC.root" "./AF4D5462-30E8-EA43-AA8A-A1EADA9796BC.root"\nxrdcp "root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv9/WZ_TuneCP5_13TeV-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/120000/936C5557-15D6-B74C-9260-582B71C8B1C3.root" "./936C5557-15D6-B74C-9260-582B71C8B1C3.root"\nxrdcp "root://t2-xrdcms.lnl.infn.it:7070//store/mc/RunIISummer20UL17NanoAODv2/GluGluToWWToTNMN_TuneCP5_13TeV_MCFM701_pythia8/NANOAODSIM/106X_mc2017_realistic_v8-v1/270000/ABA8AAB6-89A8-2A4D-B574-82CB4162656E.root" "./ABA8AAB6-89A8-2A4D-B574-82CB4162656E.root"\n'

In [3]:
'''
%%bash

ls /shared-scratch/cms/store/mc/RunIISummer20UL17NanoAODv9/ZGToLLG_01J_5f_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/AF4D5462-30E8-EA43-AA8A-A1EADA9796BC.root
ls /scratch/cms/store/mc/RunIISummer20UL17NanoAODv9/ZGToLLG_01J_5f_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/AF4D5462-30E8-EA43-AA8A-A1EADA9796BC.root
'''

'\n%%bash\n\nls /shared-scratch/cms/store/mc/RunIISummer20UL17NanoAODv9/ZGToLLG_01J_5f_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/AF4D5462-30E8-EA43-AA8A-A1EADA9796BC.root\nls /scratch/cms/store/mc/RunIISummer20UL17NanoAODv9/ZGToLLG_01J_5f_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/AF4D5462-30E8-EA43-AA8A-A1EADA9796BC.root\n'

In [39]:
from collections import defaultdict

branch_basket_counts = defaultdict(int)
branch_basket_sizes = defaultdict(int)

f = ROOT.TFile("/shared-scratch/cms/store/mc/RunIISummer20UL17NanoAODv9/ZGToLLG_01J_5f_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/AF4D5462-30E8-EA43-AA8A-A1EADA9796BC.root")

tree = f.Get("Events")

for branch in tree.GetListOfBranches():
    count = branch.GetListOfBaskets().GetSize()
    branch_basket_counts[branch.GetName()] += count
    baskets = branch.GetListOfBaskets()
    # Reset total size for this branch
    total_basket_size = 0
    for basket in baskets:
        basket_data = basket.GetBasketBytes()
        basket_size = len(basket_data)  # The size of the buffer in bytes
        total_basket_size += basket_size
    
    branch_basket_sizes[branch.GetName()] += total_basket_size

print(f"basket count: {sum(branch_basket_counts.values())}")
print(f"baskets size: {sum(branch_basket_sizes.values())}")

basket count: 16860
baskets size: 0


In [16]:
# For now I use only the branches of the first "chain", the other two are negligible but can be added if needed
file = ROOT.TFile("/shared-scratch/cms/store/mc/RunIISummer20UL17NanoAODv9/ZGToLLG_01J_5f_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/AF4D5462-30E8-EA43-AA8A-A1EADA9796BC.root")

# Access the TTree in the ROOT file
tree = file.Get("Events")

branchesss = tree.GetListOfBranches()
branchlist = []
for b in branchesss:
    branchlist.append(b.GetName())

print(len(branchlist))
print(branchlist)

def count_dataset_baskets():
    import ROOT
    import glob
    from collections import defaultdict
    
    tree_name = "Events"
    
    
    # Recursively find all .root files
    files = glob.glob("/shared-scratch/**/*.root", recursive=True)
    
    for file_path in files:
        f = ROOT.TFile.Open(file_path)
        if not f or f.IsZombie():
            continue
        tree = f.Get(tree_name)
        if not tree:
            f.Close()
            continue
        for branch in tree.GetListOfBranches():
            count = branch.GetListOfBaskets().GetSize()
            branch_basket_counts[branch.GetName()] += count
        f.Close()
    
    return sum(branch_basket_counts.values())

def dataset_baskets_size():
    import ROOT
    import glob
    from collections import defaultdict
    
    tree_name = "Events"
    branch_basket_sizes = defaultdict(int)
    
    # Recursively find all .root files
    files = glob.glob("/shared-scratch/**/*.root", recursive=True)
    
    for file_path in files:
        f = ROOT.TFile.Open(file_path)
        if not f or f.IsZombie():
            continue
        tree = f.Get(tree_name)
        if not tree:
            f.Close()
            continue
        for branch in tree.GetListOfBranches():
            baskets = branch.GetListOfBaskets()
            for basket in baskets:
                basket_size = basket.GetByteCount()
                branch_basket_sizes[branch.GetName()] += basket_size 
        f.Close()
    
    return sum(branch_basket_sizes.values())

# Edit to modify the number of branches read
#branchlist_size = len(branchlist)

# Browse the branches
#for branch in tree.GetListOfBranches():
#    print(branch.GetName())  # Print the name of each branch

1666
['run', 'luminosityBlock', 'event', 'HTXS_Higgs_pt', 'HTXS_Higgs_y', 'HTXS_stage1_1_cat_pTjet25GeV', 'HTXS_stage1_1_cat_pTjet30GeV', 'HTXS_stage1_1_fine_cat_pTjet25GeV', 'HTXS_stage1_1_fine_cat_pTjet30GeV', 'HTXS_stage1_2_cat_pTjet25GeV', 'HTXS_stage1_2_cat_pTjet30GeV', 'HTXS_stage1_2_fine_cat_pTjet25GeV', 'HTXS_stage1_2_fine_cat_pTjet30GeV', 'HTXS_stage_0', 'HTXS_stage_1_pTjet25', 'HTXS_stage_1_pTjet30', 'HTXS_njets25', 'HTXS_njets30', 'nboostedTau', 'boostedTau_chargedIso', 'boostedTau_eta', 'boostedTau_leadTkDeltaEta', 'boostedTau_leadTkDeltaPhi', 'boostedTau_leadTkPtOverTauPt', 'boostedTau_mass', 'boostedTau_neutralIso', 'boostedTau_phi', 'boostedTau_photonsOutsideSignalCone', 'boostedTau_pt', 'boostedTau_puCorr', 'boostedTau_rawAntiEle2018', 'boostedTau_rawIso', 'boostedTau_rawIsodR03', 'boostedTau_rawMVAnewDM2017v2', 'boostedTau_rawMVAoldDM2017v2', 'boostedTau_rawMVAoldDMdR032017v2', 'boostedTau_charge', 'boostedTau_decayMode', 'boostedTau_jetIdx', 'boostedTau_rawAntiEleCat2

In [14]:
client.run_on_scheduler(count_dataset_baskets)

273326476

In [17]:
client.run_on_scheduler(dataset_baskets_size)

0

In [5]:
#from my_initialization_function_UL import *
text_file = open("preselection_UL.h", "r")
data = text_file.read()

text_file = open("preselection_part2_UL.h", "r")
data_2 = text_file.read()

def my_initialization_function():
    
    import ROOT
    import sys
    ROOT.EnableThreadSafety()
    #ROOT.gInterpreter.AddIncludePath("/cvmfs/cms.dodas.infn.it/boost_1_77_0")
    sys.path.append("/lib/python3.8/site-packages")
    ROOT.gInterpreter.AddIncludePath("/usr/lib/boost_1_77_0")
    ROOT.gInterpreter.AddIncludePath("/opt/conda/include")
    
    jec_prefix_MC = "Summer19UL17_V6_MC"
    jer_prefix_MC = "Summer19UL17_JRV3_MC"
    
    from urllib import request 
    request.urlretrieve("https://ttedesch.web.cern.ch/ttedesch/NEWERA/nanoAOD-tools/python/postprocessing/data/roccor.Run2.v5/RoccoR2017UL.txt", "RoccoR2017UL.txt")
    request.urlretrieve("https://ttedesch.web.cern.ch/ttedesch/NEWERA/nanoAOD-tools/python/postprocessing/data/roccor.Run2.v5/RoccoR.cc", "RoccoR.cc")
    request.urlretrieve("https://ttedesch.web.cern.ch/ttedesch/NEWERA/nanoAOD-tools/python/postprocessing/data/roccor.Run2.v5/RoccoR.h", "RoccoR.h")
    request.urlretrieve("https://ttedesch.web.cern.ch/ttedesch/NEWERA/nanoAOD-tools/data/btagSF/DeepJet_106XUL17_v3_new.csv", "DeepJet_106XUL17_v3_new.csv") 
    
    
    ROOT.gInterpreter.Declare(
        '''
        #ifndef ROCCOR
        #define ROCCOR
        #include "RoccoR.cc"
        #endif
        ''')

    request.urlretrieve("https://ttedesch.web.cern.ch/ttedesch/NEWERA/nanoAOD-tools/data/jme/{}_L1FastJet_AK4PFchs.txt".format(jec_prefix_MC), "{}_L1FastJet_AK4PFchs.txt".format(jec_prefix_MC))
    request.urlretrieve("https://ttedesch.web.cern.ch/ttedesch/NEWERA/nanoAOD-tools/data/jme/{}_L2Relative_AK4PFchs.txt".format(jec_prefix_MC), "{}_L2Relative_AK4PFchs.txt".format(jec_prefix_MC)) 
    request.urlretrieve("https://ttedesch.web.cern.ch/ttedesch/NEWERA/nanoAOD-tools/data/jme/{}_L3Absolute_AK4PFchs.txt".format(jec_prefix_MC), "{}_L3Absolute_AK4PFchs.txt".format(jec_prefix_MC)) 
    request.urlretrieve("https://ttedesch.web.cern.ch/ttedesch/NEWERA/nanoAOD-tools/data/jme/{}_L2L3Residual_AK4PFchs.txt".format(jec_prefix_MC), "{}_L2L3Residual_AK4PFchs.txt".format(jec_prefix_MC)) 
    request.urlretrieve("https://ttedesch.web.cern.ch/ttedesch/NEWERA/nanoAOD-tools/data/jme/{}_UncertaintySources_AK4PFchs.txt".format(jec_prefix_MC), "{}_UncertaintySources_AK4PFchs.txt".format(jec_prefix_MC)) 
    request.urlretrieve("https://ttedesch.web.cern.ch/ttedesch/NEWERA/nanoAOD-tools/data/jme/{}_PtResolution_AK4PFchs.txt".format(jer_prefix_MC), "{}_PtResolution_AK4PFchs.txt".format(jer_prefix_MC))
    request.urlretrieve("https://ttedesch.web.cern.ch/ttedesch/NEWERA/nanoAOD-tools/data/jme/{}_SF_AK4PFchs.txt".format(jer_prefix_MC), "{}_SF_AK4PFchs.txt".format(jer_prefix_MC))
    
    
    ROOT.gInterpreter.Declare('{}'.format(data))

    ROOT.gInterpreter.ProcessLine('#ifndef LOADING')
    ROOT.gInterpreter.ProcessLine('#define LOADING')
    
    ROOT.gInterpreter.ProcessLine('reader_1_UL2017.load(calibration_UL2017, BTagEntry::FLAV_B, "comb");')
    ROOT.gInterpreter.ProcessLine('reader_1_UL2017.load(calibration_UL2017, BTagEntry::FLAV_C, "comb");')
    ROOT.gInterpreter.ProcessLine('reader_1_UL2017.load(calibration_UL2017, BTagEntry::FLAV_UDSG, "incl");')

    ROOT.gInterpreter.ProcessLine('#endif')
    
    ROOT.gInterpreter.Declare('{}'.format(data_2))
    

    from CMSJMECalculators import loadJMESystematicsCalculators
    loadJMESystematicsCalculators()

    
    ################################# JET #####################################################################################################
    
    ROOT.gROOT.ProcessLine("JetVariationsCalculator myJetVarCalc{};")
    calc = getattr(ROOT, "myJetVarCalc")

    # redo JEC, push_back corrector parameters for different levels
    jecParams = getattr(ROOT, "std::vector<JetCorrectorParameters>")()
    jecParams.push_back(ROOT.JetCorrectorParameters("./{}_L1FastJet_AK4PFchs.txt".format(jec_prefix_MC)))
    jecParams.push_back(ROOT.JetCorrectorParameters("./{}_L2Relative_AK4PFchs.txt".format(jec_prefix_MC)))
    #jecParams.push_back(ROOT.JetCorrectorParameters("./{}_L3Absolute_AK4PFchs.txt".format(jec_prefix_MC)))
    jecParams.push_back(ROOT.JetCorrectorParameters("./{}_L2L3Residual_AK4PFchs.txt".format(jec_prefix_MC)))
    calc.setJEC(jecParams)

    uncert_sources = ["Total"]
    #uncert_sources = []
    # calculate JES uncertainties
    with open("./{}_UncertaintySources_AK4PFchs.txt".format(jec_prefix_MC)) as f:
        lines = f.read().split("\n")
        sources = [
            x for x in lines if x.startswith("[") and x.endswith("]")
        ]
        sources = [x[1:-1] for x in sources]
        sources = list(filter(lambda source: source in uncert_sources, sources))

    for s in sources:
        jcp_unc = ROOT.JetCorrectorParameters("./{}_UncertaintySources_AK4PFchs.txt".format(jec_prefix_MC), s)
        calc.addJESUncertainty(s, jcp_unc)

    # Smear jets, with JER uncertainty
    calc.setSmearing("./{}_PtResolution_AK4PFchs.txt".format(jer_prefix_MC), "./{}_SF_AK4PFchs.txt".format(jer_prefix_MC), False, True, 0.2, 3.)  # use hybrid recipe, matching parameters

    jetvariations = [calc.available()[i] for i in range(0,calc.available().size())]
    
    ############################################ MET T1SMEAR ###########################################################################################
    
    ROOT.gROOT.ProcessLine("Type1METVariationsCalculator myType1SmearMETVarCalc{};")
    calc = getattr(ROOT, "myType1SmearMETVarCalc")

    # redo JEC, push_back corrector parameters for different levels
    jecParams = getattr(ROOT, "std::vector<JetCorrectorParameters>")()
    jecParams.push_back(ROOT.JetCorrectorParameters("./{}_L1FastJet_AK4PFchs.txt".format(jec_prefix_MC)))
    jecParams.push_back(ROOT.JetCorrectorParameters("./{}_L2Relative_AK4PFchs.txt".format(jec_prefix_MC)))
    jecParams.push_back(ROOT.JetCorrectorParameters("./{}_L2L3Residual_AK4PFchs.txt".format(jec_prefix_MC)))

    calc.setJEC(jecParams)

    jecParams = getattr(ROOT, "std::vector<JetCorrectorParameters>")()
    jecParams.push_back(ROOT.JetCorrectorParameters("./{}_L1FastJet_AK4PFchs.txt".format(jec_prefix_MC)))
    calc.setL1JEC(jecParams)

    calc.setIsT1SmearedMET(True)

    # calculate JES uncertainties
    with open("./{}_UncertaintySources_AK4PFchs.txt".format(jec_prefix_MC)) as f:
        lines = f.read().split("\n")
        sources = [
            x for x in lines if x.startswith("[") and x.endswith("]")
        ]
        sources = [x[1:-1] for x in sources]
        sources = list(filter(lambda source: source in uncert_sources, sources))


    for s in sources:
        jcp_unc = ROOT.JetCorrectorParameters("./{}_UncertaintySources_AK4PFchs.txt".format(jec_prefix_MC), s)
        calc.addJESUncertainty(s, jcp_unc)

    # Smear jets, with JER uncertainty
    calc.setSmearing("./{}_PtResolution_AK4PFchs.txt".format(jer_prefix_MC), "./{}_SF_AK4PFchs.txt".format(jer_prefix_MC), False, True, 0.2, 3.)  # use hybrid recipe, matching parameters

    mett1smearvariations = [calc.available()[i] for i in range(0, calc.available().size())]
    
    print(mett1smearvariations)

    #ROOT.gInterpreter.ProcessLine('#endif')

    
jetvariations = [
 'nominal',
 'jerUp',
 'jerDown',
 'jesTotalUp',
 'jesTotalDown',
]

mett1smearvariations = jetvariations

if distributed == True:
    RDataFrame = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame
    client = Client(address="tcp://127.0.0.1:"+str(sched_port))
    client.restart()
    try:
        client.register_plugin(UploadFile("/opt/workspace/persistent-storage/proxy"))
    except:
        pass
    client.run(set_proxy)
    ROOT.RDF.Experimental.Distributed.initialize(my_initialization_function)
    print(f"Memory info pre run:\n" + client.run_on_scheduler(get_memory_info))
    print(f"Folder size:\n" + client.run_on_scheduler(dir_size))
    print(f"/proc/diskstats pre run:\n" + client.run_on_scheduler(print_proc_diskstats))

else:
    RDataFrame = ROOT.RDataFrame
    my_initialization_function()

['nominal', 'jerup', 'jerdown', 'jesTotalup', 'jesTotaldown', 'unclustEnup', 'unclustEndown']
Memory info pre run:
               total        used        free      shared  buff/cache   available
Mem:           1.5Ti        33Gi       1.4Ti       582Mi       9.2Gi       1.4Ti
Swap:           31Gi       7.8Mi        31Gi

Folder size:
4.8T	/scratch
/proc/diskstats pre run:
 259       0 nvme2n1 70352583 32096717 11887167909 29787372 29046368 519212705 4376258157 3780055 0 6154783 33567428 0 0 0 0 0 0
 259       1 nvme2n1p1 70352485 32096717 11887165237 29787365 29046368 519212705 4376258157 3780055 0 7917171 33567421 0 0 0 0 0 0
 259       2 nvme0n1 70364618 32048561 11889465632 29772331 29023070 519337923 4377073517 4411791 0 6759295 34184123 0 0 0 0 0 0
 259       6 nvme0n1p1 70364520 32048561 11889462960 29772323 29023070 519337923 4377073517 4411791 0 8582912 34184115 0 0 0 0 0 0
 259       3 nvme4n1 70406794 32157474 11881125970 29804969 28963121 519194772 4375448717 3955004 0 61954

Error in <TCling::LoadPCM>: ROOT PCM /opt/conda/lib/G__CMSJMECalculators_rdict.pcm file does not exist
Info in <TCling::LoadPCM>: In-memory ROOT PCM candidate /usr/local/lib/root/libASImageGui_rdict.pcm

Info in <TCling::LoadPCM>: In-memory ROOT PCM candidate /usr/local/lib/root/libASImage_rdict.pcm

Info in <TCling::LoadPCM>: In-memory ROOT PCM candidate /usr/local/lib/root/libEG_rdict.pcm

Info in <TCling::LoadPCM>: In-memory ROOT PCM candidate /usr/local/lib/root/libFitPanel_rdict.pcm

Info in <TCling::LoadPCM>: In-memory ROOT PCM candidate /usr/local/lib/root/libFoam_rdict.pcm

Info in <TCling::LoadPCM>: In-memory ROOT PCM candidate /usr/local/lib/root/libFumili_rdict.pcm

Info in <TCling::LoadPCM>: In-memory ROOT PCM candidate /usr/local/lib/root/libGX11TTF_rdict.pcm

Info in <TCling::LoadPCM>: In-memory ROOT PCM candidate /usr/local/lib/root/libGX11_rdict.pcm

Info in <TCling::LoadPCM>: In-memory ROOT PCM candidate /usr/local/lib/root/libGed_rdict.pcm

Info in <TCling::LoadPCM>: 

In [6]:
if distributed == True:
    RDataFrame = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame
    client = Client(address="tcp://127.0.0.1:"+str(sched_port))
    client.restart()
    try:
        client.register_plugin(UploadFile("/opt/workspace/persistent-storage/proxy"))
    except:
        pass
    client.run(set_proxy)
    ROOT.RDF.Experimental.Distributed.initialize(my_initialization_function)
    print(f"Memory info pre run:\n" + client.run_on_scheduler(get_memory_info))
    print(f"Folder size:\n" + client.run_on_scheduler(dir_size))
    print(f"/proc/diskstats pre run:\n" + client.run_on_scheduler(print_proc_diskstats))

else:
    RDataFrame = ROOT.RDataFrame
    my_initialization_function()

['nominal', 'jerup', 'jerdown', 'jesTotalup', 'jesTotaldown', 'unclustEnup', 'unclustEndown']
Memory info pre run:
               total        used        free      shared  buff/cache   available
Mem:           1.5Ti        33Gi       1.4Ti       582Mi       9.2Gi       1.4Ti
Swap:           31Gi       7.8Mi        31Gi

Folder size:
4.8T	/scratch
/proc/diskstats pre run:
 259       0 nvme2n1 70352687 32096961 11887170693 29787386 29046826 519213839 4376270823 3780073 0 6154798 33567459 0 0 0 0 0 0
 259       1 nvme2n1p1 70352589 32096961 11887168021 29787379 29046826 519213839 4376270823 3780073 0 7917189 33567453 0 0 0 0 0 0
 259       2 nvme0n1 70364729 32048837 11889468728 29772345 29023505 519338908 4377084807 4411807 0 6759310 34184153 0 0 0 0 0 0
 259       6 nvme0n1p1 70364631 32048837 11889466056 29772338 29023505 519338908 4377084807 4411807 0 8582932 34184146 0 0 0 0 0 0
 259       3 nvme4n1 70406934 32157808 11881129762 29804989 28963540 519195694 4375459375 3955021 0 61954

In [7]:
sampleDict = {'ZZtoLep_UL2016APV': 0,'ZZTo2L2Nu_UL2016APV': 1,'ZZTo4L_UL2016APV': 2,'GluGluToContinToZZTo4e_UL2016APV': 3,'GluGluToContinToZZTo2e2mu_UL2016APV': 4,'GluGluToContinToZZTo2e2tau_UL2016APV': 5,'GluGluToContinToZZTo2mu2nu_UL2016APV': 6,'GluGluToContinToZZTo4mu_UL2016APV': 7,'GluGluToContinToZZTo2mu2tau_UL2016APV': 8,'GluGluToContinToZZTo2tau2nu_UL2016APV': 9,'GluGluToContinToZZTo4tau_UL2016APV': 10,'GluGluToContinToZZTo2e2nu_UL2016APV': 11,'TT_UL2016APV': 12,'TT_SemiLep_UL2016APV': 13,'TT_Had_UL2016APV': 14,'TTTo2L2Nu_UL2016APV': 15,'TT_beff_UL2016APV': 16,'TVX_UL2016APV': 17,'TTGJets_UL2016APV': 18,'TTZToQQ_UL2016APV': 19,'TTZToLLNuNu_UL2016APV': 20,'TTWJetsToQQ_UL2016APV': 21,'TTWJetsToLNu_UL2016APV': 22,'tZq_ll_4f_UL2016APV': 23,'VG_UL2016APV': 24,'ZG_UL2016APV': 25,'WG_UL2016APV': 26,'WrongSign_UL2016APV': 27,'WWto2L2Nu_UL2016APV': 28,'GluGluToWWToENEN_UL2016APV': 29,'GluGluToWWToENMN_UL2016APV': 30,'GluGluToWWToENTN_UL2016APV': 31,'GluGluToWWToMNEN_UL2016APV': 32,'GluGluToWWToMNMN_UL2016APV': 33,'GluGluToWWToMNTN_UL2016APV': 34,'GluGluToWWToTNEN_UL2016APV': 35,'GluGluToWWToTNMN_UL2016APV': 36,'GluGluToWWToTNTN_UL2016APV': 37,'ST_tW_top_UL2016APV': 38,'ST_tW_antitop_UL2016APV': 39,'GluGluHToWWTo2L2Nu_UL2016APV': 40,'GluGluHToWWToLNuQQ_UL2016APV': 41,'GluGluHToZZTo4L_UL2016APV': 42,'GluGluHToTauTau_UL2016APV': 43,'VBFHToWWTo2L2Nu_UL2016APV': 44,'VBFHToTauTau_UL2016APV': 45,'ttHToNonbb_UL2016APV': 46,'VHToNonbb_UL2016APV': 47,'Triboson_UL2016APV': 48,'WWTo2L2Nu_DoubleScattering_UL2016': 49,'WWW_4F_UL2016APV': 50,'WWZ_4F_UL2016APV': 51,'WZZ_UL2016APV': 52,'ZZZ_UL2016APV': 53,'WWG_UL2016APV': 54,'WJets_UL2016APV': 55,'WJetsHT70to100_UL2016APV': 56,'WJetsHT100to200_UL2016APV': 57,'WJetsHT200to400_UL2016APV': 58,'WJetsHT400to600_UL2016APV': 59,'WJetsHT600to800_UL2016APV': 60,'WJetsHT800to1200_UL2016APV': 61,'WJetsHT1200to2500_UL2016APV': 62,'WJetsHT2500toInf_UL2016APV': 63,'WZ_UL2016APV': 64,'DYJetsToLL_UL2016APV': 65,'DYJetsToLL_FxFx_UL2016APV': 65,'DYJetsToLL_M10to50_UL2016APV': 66,'DYJetsToLL_M50_UL2016APV': 67,'DYJetsToLL_M50_FxFx_UL2016APV': 67,'DYJetsToLL_M50_UL2016APV_ext': 67,'WpWpJJ_EWK_UL2016APV': 68,'WpWpJJ_QCD_UL2016APV': 69,'VBS_SSWW_SM_UL2016APV': 70,'VBS_SSWW_LL_SM_UL2016APV': 71,'VBS_SSWW_TL_SM_UL2016APV': 72,'VBS_SSWW_TT_SM_UL2016APV': 73,'VBS_SSWW_cW_UL2016APV': 74,'VBS_SSWW_cW_SM_UL2016APV': 75,'VBS_SSWW_cW_BSM_UL2016APV': 76,'VBS_SSWW_cW_INT_UL2016APV': 77,'VBS_SSWW_cHW_UL2016APV': 78,'VBS_SSWW_cHW_SM_UL2016APV': 79,'VBS_SSWW_cHW_BSM_UL2016APV': 80,'VBS_SSWW_cHW_INT_UL2016APV': 81,'VBS_SSWW_cW_cHW_UL2016APV': 82,'VBS_SSWW_DIM6_UL2016APV': 83,'VBS_SSWW_DIM6_SM_UL2016APV': 84,'ZZtoLep_UL2016': 85,'ZZTo2L2Nu_UL2016': 86,'ZZTo4L_UL2016': 87,'GluGluToContinToZZTo4e_UL2016': 88,'GluGluToContinToZZTo2e2mu_UL2016': 89,'GluGluToContinToZZTo2e2tau_UL2016': 90,'GluGluToContinToZZTo2mu2nu_UL2016': 91,'GluGluToContinToZZTo4mu_UL2016': 92,'GluGluToContinToZZTo2mu2tau_UL2016': 93,'GluGluToContinToZZTo2tau2nu_UL2016': 94,'GluGluToContinToZZTo4tau_UL2016': 95,'GluGluToContinToZZTo2e2nu_UL2016': 96,'TT_UL2016': 97,'TT_SemiLep_UL2016': 98,'TT_Had_UL2016': 99,'TTTo2L2Nu_UL2016': 100,'TT_beff_UL2016': 101,'TVX_UL2016': 102,'TTGJets_UL2016': 103,'TTZToQQ_UL2016': 104,'TTZToLLNuNu_UL2016': 105,'TTWJetsToQQ_UL2016': 106,'TTWJetsToLNu_UL2016': 107,'tZq_ll_4f_UL2016': 108,'VG_UL2016': 109,'ZG_UL2016': 110,'WG_UL2016': 111,'WrongSign_UL2016': 112,'WWto2L2Nu_UL2016': 113,'GluGluToWWToENEN_UL2016': 114,'GluGluToWWToENMN_UL2016': 115,'GluGluToWWToENTN_UL2016': 116,'GluGluToWWToMNEN_UL2016': 117,'GluGluToWWToMNMN_UL2016': 118,'GluGluToWWToMNTN_UL2016': 119,'GluGluToWWToTNEN_UL2016': 120,'GluGluToWWToTNMN_UL2016': 121,'GluGluToWWToTNTN_UL2016': 122,'ST_tW_top_UL2016': 123,'ST_tW_antitop_UL2016': 124,'GluGluHToWWTo2L2Nu_UL2016': 125,'GluGluHToWWToLNuQQ_UL2016': 126,'GluGluHToZZTo4L_UL2016': 127,'GluGluHToTauTau_UL2016': 128,'VBFHToWWTo2L2Nu_UL2016': 129,'VBFHToTauTau_UL2016': 130,'ttHToNonbb_UL2016': 131,'VHToNonbb_UL2016': 132,'Triboson_UL2016': 133,'WWW_4F_UL2016': 134,'WWZ_4F_UL2016': 135,'WZZ_UL2016': 136,'ZZZ_UL2016': 137,'WWG_UL2016': 138,'WJets_UL2016': 139,'WJetsHT70to100_UL2016': 140,'WJetsHT100to200_UL2016': 141,'WJetsHT200to400_UL2016': 142,'WJetsHT400to600_UL2016': 143,'WJetsHT600to800_UL2016': 144,'WJetsHT800to1200_UL2016': 145,'WJetsHT1200to2500_UL2016': 146,'WJetsHT2500toInf_UL2016': 147,'WZ_UL2016': 148,'DYJetsToLL_UL2016': 149,'DYJetsToLL_FxFx_UL2016': 149,'DYJetsToLL_M10to50_UL2016': 150,'DYJetsToLL_M50_UL2016': 151,'DYJetsToLL_M50_FxFx_UL2016': 151,'DYJetsToLL_M50_UL2016_ext': 151,'WpWpJJ_EWK_UL2016': 152,'WpWpJJ_QCD_UL2016': 153,'VBS_SSWW_SM_UL2016': 154,'VBS_SSWW_LL_SM_UL2016': 155,'VBS_SSWW_TL_SM_UL2016': 156,'VBS_SSWW_TT_SM_UL2016': 157,'VBS_SSWW_cW_UL2016': 158,'VBS_SSWW_cW_SM_UL2016': 159,'VBS_SSWW_cW_BSM_UL2016': 160,'VBS_SSWW_cW_INT_UL2016': 161,'VBS_SSWW_cHW_UL2016': 162,'VBS_SSWW_cHW_SM_UL2016': 163,'VBS_SSWW_cHW_BSM_UL2016': 164,'VBS_SSWW_cHW_INT_UL2016': 165,'VBS_SSWW_cW_cHW_UL2016': 166,'VBS_SSWW_DIM6_UL2016': 167,'VBS_SSWW_DIM6_SM_UL2016': 168,'ZZtoLep_UL2017': 169,'ZZTo2L2Nu_UL2017': 170,'ZZTo4L_UL2017': 171,'GluGluToContinToZZTo4e_UL2017': 172,'GluGluToContinToZZTo2e2mu_UL2017': 173,'GluGluToContinToZZTo2e2tau_UL2017': 174,'GluGluToContinToZZTo2mu2nu_UL2017': 175,'GluGluToContinToZZTo4mu_UL2017': 176,'GluGluToContinToZZTo2mu2tau_UL2017': 177,'GluGluToContinToZZTo2tau2nu_UL2017': 178,'GluGluToContinToZZTo4tau_UL2017': 179,'GluGluToContinToZZTo2e2nu_UL2017': 180,'TT_UL2017': 181,'TT_SemiLep_UL2017': 182,'TT_Had_UL2017': 183,'TTTo2L2Nu_UL2017': 184,'TT_beff_UL2017': 185,'TVX_UL2017': 186,'TTGJets_UL2017': 187,'TTZToQQ_UL2017': 188,'TTZToLLNuNu_UL2017': 189,'TTWJetsToQQ_UL2017': 190,'TTWJetsToLNu_UL2017': 191,'tZq_ll_4f_UL2017': 192,'VG_UL2017': 193,'ZG_UL2017': 194,'WG_UL2017': 195,'WrongSign_UL2017': 196,'WWto2L2Nu_UL2017': 197,'GluGluToWWToENEN_UL2017': 198,'GluGluToWWToENMN_UL2017': 199,'GluGluToWWToENTN_UL2017': 200,'GluGluToWWToMNEN_UL2017': 201,'GluGluToWWToMNMN_UL2017': 202,'GluGluToWWToMNTN_UL2017': 203,'GluGluToWWToTNEN_UL2017': 204,'GluGluToWWToTNMN_UL2017': 205,'GluGluToWWToTNTN_UL2017': 206,'ST_tW_top_UL2017': 207,'ST_tW_antitop_UL2017': 208,'GluGluHToWWTo2L2Nu_UL2017': 209,'GluGluHToWWToLNuQQ_UL2017': 210,'GluGluHToZZTo4L_UL2017': 211,'GluGluHToTauTau_UL2017': 212,'VBFHToWWTo2L2Nu_UL2017': 213,'VBFHToTauTau_UL2017': 214,'ttHToNonbb_UL2017': 215,'VHToNonbb_UL2017': 216,'Triboson_UL2017': 217,'WWTo2L2Nu_DoubleScattering_UL2017': 218,'WWW_4F_UL2017': 219,'WWZ_4F_UL2017': 220,'WZZ_UL2017': 221,'ZZZ_UL2017': 222,'WWG_UL2017': 223,'WJets_UL2017': 224,'WJetsHT70to100_UL2017': 225,'WJetsHT100to200_UL2017': 226,'WJetsHT200to400_UL2017': 227,'WJetsHT400to600_UL2017': 228,'WJetsHT600to800_UL2017': 229,'WJetsHT800to1200_UL2017': 230,'WJetsHT1200to2500_UL2017': 231,'WJetsHT2500toInf_UL2017': 232,'WZ_UL2017': 233,'DYJetsToLL_UL2017': 234,'DYJetsToLL_FxFx_UL2017': 234,'DYJetsToLL_M10to50_UL2017': 235,'DYJetsToLL_M50_UL2017': 236,'DYJetsToLL_M50_FxFx_UL2017': 236,'DYJetsToLL_M50_UL2017_ext': 236,'WpWpJJ_EWK_UL2017': 237,'WpWpJJ_QCD_UL2017': 238,'VBS_SSWW_SM_UL2017': 239,'VBS_SSWW_LL_SM_UL2017': 240,'VBS_SSWW_TL_SM_UL2017': 241,'VBS_SSWW_TT_SM_UL2017': 242,'VBS_SSWW_cW_UL2017': 243,'VBS_SSWW_cW_SM_UL2017': 244,'VBS_SSWW_cW_BSM_UL2017': 245,'VBS_SSWW_cW_INT_UL2017': 246,'VBS_SSWW_cHW_UL2017': 247,'VBS_SSWW_cHW_SM_UL2017': 248,'VBS_SSWW_cHW_BSM_UL2017': 249,'VBS_SSWW_cHW_INT_UL2017': 250,'VBS_SSWW_cW_cHW_UL2017': 251,'VBS_SSWW_DIM6_UL2017': 252,'VBS_SSWW_DIM6_SM_UL2017': 253,'ZZtoLep_UL2018': 254,'ZZTo2L2Nu_UL2018': 255,'ZZTo4L_UL2018': 256,'GluGluToContinToZZTo4e_UL2018': 257,'GluGluToContinToZZTo2e2mu_UL2018': 258,'GluGluToContinToZZTo2e2tau_UL2018': 259,'GluGluToContinToZZTo2mu2nu_UL2018': 260,'GluGluToContinToZZTo4mu_UL2018': 261,'GluGluToContinToZZTo2mu2tau_UL2018': 262,'GluGluToContinToZZTo2tau2nu_UL2018': 263,'GluGluToContinToZZTo4tau_UL2018': 264,'GluGluToContinToZZTo2e2nu_UL2018': 265,'TT_UL2018': 266,'TT_SemiLep_UL2018': 267,'TT_Had_UL2018': 268,'TTTo2L2Nu_UL2018': 269,'TT_beff_UL2018': 270,'TVX_UL2018': 271,'TTGJets_UL2018': 272,'TTZToQQ_UL2018': 273,'TTZToLLNuNu_UL2018': 274,'TTWJetsToQQ_UL2018': 275,'TTWJetsToLNu_UL2018': 276,'tZq_ll_4f_UL2018': 277,'VG_UL2018': 278,'ZG_UL2018': 279,'WG_UL2018': 280,'WrongSign_UL2018': 281,'WWto2L2Nu_UL2018': 282,'GluGluToWWToENEN_UL2018': 283,'GluGluToWWToENMN_UL2018': 284,'GluGluToWWToENTN_UL2018': 285,'GluGluToWWToMNEN_UL2018': 286,'GluGluToWWToMNMN_UL2018': 287,'GluGluToWWToMNTN_UL2018': 288,'GluGluToWWToTNEN_UL2018': 289,'GluGluToWWToTNMN_UL2018': 290,'GluGluToWWToTNTN_UL2018': 291,'ST_tW_top_UL2018': 292,'ST_tW_antitop_UL2018': 293,'GluGluHToWWTo2L2Nu_UL2018': 294,'GluGluHToWWToLNuQQ_UL2018': 295,'GluGluHToZZTo4L_UL2018': 296,'GluGluHToTauTau_UL2018': 297,'VBFHToWWTo2L2Nu_UL2018': 298,'VBFHToTauTau_UL2018': 299,'ttHToNonbb_UL2018': 300,'VHToNonbb_UL2018': 301,'Triboson_UL2018': 302,'WWTo2L2Nu_DoubleScattering_UL2018': 303,'WWW_4F_UL2018': 304,'WWZ_4F_UL2018': 305,'WZZ_UL2018': 306,'ZZZ_UL2018': 307,'WWG_UL2018': 308,'WJets_UL2018': 309,'WJetsHT70to100_UL2018': 310,'WJetsHT100to200_UL2018': 311,'WJetsHT200to400_UL2018': 312,'WJetsHT400to600_UL2018': 313,'WJetsHT600to800_UL2018': 314,'WJetsHT800to1200_UL2018': 315,'WJetsHT1200to2500_UL2018': 316,'WJetsHT2500toInf_UL2018': 317,'WZ_UL2018': 318,'DYJetsToLL_UL2018': 319,'DYJetsToLL_FxFx_UL2018': 319,'DYJetsToLL_M10to50_UL2018': 320,'DYJetsToLL_M50_UL2018': 321,'DYJetsToLL_M50_FxFx_UL2018': 321,'DYJetsToLL_M50_UL2018_ext': 321,'WpWpJJ_EWK_UL2018': 322,'WpWpJJ_QCD_UL2018': 323,'VBS_SSWW_SM_UL2018': 324,'VBS_SSWW_LL_SM_UL2018': 325,'VBS_SSWW_TL_SM_UL2018': 326,'VBS_SSWW_TT_SM_UL2018': 327,'VBS_SSWW_cW_UL2018': 328,'VBS_SSWW_cW_BSM_UL2018': 329,'VBS_SSWW_cW_SM_UL2018': 330,'VBS_SSWW_cW_INT_UL2018': 331,'VBS_SSWW_cHW_UL2018': 332,'VBS_SSWW_cHW_SM_UL2018': 333,'VBS_SSWW_cHW_BSM_UL2018': 334,'VBS_SSWW_cHW_INT_UL2018': 335,'VBS_SSWW_cW_cHW_UL2018': 336,'VBS_SSWW_DIM6_UL2018': 337,'VBS_SSWW_DIM6_SM_UL2018': 338,'DataMu_UL2016APV': 339,'DataMuB1_UL2016APV': 340,'DataMuB2_UL2016APV': 341,'DataMuC_UL2016APV': 342,'DataMuD_UL2016APV': 343,'DataMuE_UL2016APV': 344,'DataMuF_UL2016APV': 345,'DataMu_UL2016': 346,'DataMuF_UL2016': 347,'DataMuG_UL2016': 348,'DataMuH_UL2016': 349,'DataMu_UL2017': 350,'DataMuB_UL2017': 351,'DataMuC_UL2017': 352,'DataMuD_UL2017': 353,'DataMuE_UL2017': 354,'DataMuF_UL2017': 355,'DataMu_UL2018': 356,'DataMuA_UL2018': 357,'DataMuB_UL2018': 358,'DataMuC_UL2018': 359,'DataMuD_UL2018': 360,'DataEle_UL2016APV': 361,'DataEleB1_UL2016APV': 362,'DataEleB2_UL2016APV': 363,'DataEleC_UL2016APV': 364,'DataEleD_UL2016APV': 365,'DataEleE_UL2016APV': 366,'DataEleF_UL2016APV': 367,'DataEle_UL2016': 368,'DataEleF_UL2016': 369,'DataEleG_UL2016': 370,'DataEleH_UL2016': 371,'DataEle_UL2017': 372,'DataEleB_UL2017': 373,'DataEleC_UL2017': 374,'DataEleD_UL2017': 375,'DataEleE_UL2017': 376,'DataEleF_UL2017': 377,'DataEle_UL2018': 378,'DataEleA_UL2018': 379,'DataEleB_UL2018': 380,'DataEleC_UL2018': 381,'DataEleD_UL2018': 382,'DataHT_UL2016APV': 383,'DataHTB1_UL2016APV': 384,'DataHTB2_UL2016APV': 385,'DataHTC_UL2016APV': 386,'DataHTD_UL2016APV': 387,'DataHTE_UL2016APV': 388,'DataHTF_UL2016APV': 389,'DataHT_UL2016': 390,'DataHTF_UL2016': 391,'DataHTG_UL2016': 392,'DataHTH_UL2016': 393,'DataHT_UL2017': 394,'DataHTB_UL2017': 395,'DataHTC_UL2017': 396,'DataHTD_UL2017': 397,'DataHTE_UL2017': 398,'DataHTF_UL2017': 399,'DataHT_UL2018': 400,'DataHTA_UL2018': 401,'DataHTB_UL2018': 402,'DataHTC_UL2018': 403,'DataHTD_UL2018': 404,'SampleHTFake_UL2016APV': 405,'SampleHTFake_UL2016': 406,'SampleHTFake_UL2017': 407,'SampleHTFake_UL2018': 408,}

aggregated_samples_UL2017 = {
    'VG':  [ZG_UL2017, WG_UL2017],
    'TVX': [TTGJets_UL2017, TTZToQQ_UL2017, TTZToLLNuNu_UL2017, TTWJetsToQQ_UL2017, TTWJetsToLNu_UL2017, tZq_ll_4f_UL2017], 
    'Triboson': [WWW_4F_UL2017, WWZ_4F_UL2017, WZZ_UL2017, ZZZ_UL2017, WWG_UL2017], 
    'TTTo2L2Nu': [TTTo2L2Nu_UL2017], 
    'WZ': [WZ_UL2017], 
    'DYJetsToLL_FxFx': [DYJetsToLL_M50_FxFx_UL2017],
    'WrongSign': [WWto2L2Nu_UL2017, GluGluToWWToENEN_UL2017, GluGluToWWToENMN_UL2017, GluGluToWWToENTN_UL2017, GluGluToWWToMNEN_UL2017, GluGluToWWToMNMN_UL2017, GluGluToWWToMNTN_UL2017, GluGluToWWToTNEN_UL2017, 
                  GluGluToWWToTNMN_UL2017, 
                  GluGluToWWToTNTN_UL2017, ST_tW_top_UL2017, 
                  ST_tW_antitop_UL2017, 
                  GluGluHToWWTo2L2Nu_UL2017, 
                  GluGluHToZZTo4L_UL2017, GluGluHToTauTau_UL2017, VBFHToWWTo2L2Nu_UL2017, VBFHToTauTau_UL2017, ttHToNonbb_UL2017, VHToNonbb_UL2017
    ], 
    'ZZtoLep': [ZZTo2L2Nu_UL2017, ZZTo4L_UL2017, GluGluToContinToZZTo2e2nu_UL2017, GluGluToContinToZZTo2e2mu_UL2017, GluGluToContinToZZTo2e2tau_UL2017, GluGluToContinToZZTo2mu2nu_UL2017, GluGluToContinToZZTo2mu2tau_UL2017, GluGluToContinToZZTo4e_UL2017, GluGluToContinToZZTo4mu_UL2017, GluGluToContinToZZTo4tau_UL2017], 
    'VBS_SSWW_SM': [VBS_SSWW_LL_SM_UL2017, VBS_SSWW_TL_SM_UL2017, VBS_SSWW_TT_SM_UL2017], 
}

aggregated_samples = aggregated_samples_UL2017

In [8]:
from rucio.client.client import Client
import os

import logging
logger = logging.getLogger(__name__)
logging.disable('WARNING')

os.environ["RUCIO_HOME"] = "/cvmfs/cms.cern.ch/rucio/current/"
os.environ['X509_CERT_DIR'] = "/cvmfs/grid.cern.ch/etc/grid-security/certificates/"
os.environ['X509_USER_PROXY'] = "/opt/workspace/persistent-storage/proxy"
RUCIO_SCOPE = "cms"
ACCOUNT = "lpaciose" #"ttedesch"

SITENAME = "T2_IT_Legnaro"

c = Client(account=ACCOUNT, auth_type="x509_proxy")

chain = []
chain_gluglu = []
chain_WZ = []

proxy = redirector
max_lines = maxNfilespersample

for k,v in aggregated_samples.items():
    for s in v:
        CMS_DATASET_NAME = s.dataset
        files = c.list_files(scope=RUCIO_SCOPE, name=CMS_DATASET_NAME)
        lfns = []
        for f in files:
            lfns.append(RUCIO_SCOPE + ':' + f['name'])
        for i,line in enumerate(lfns):
            if i < max_lines and not line[4:] in []:
                if k != 'Data':
                    if s.name in ["GluGluToWWToTNMN_UL2017", "GluGluHToWWTo2L2Nu_UL2017"]:
                        chain_gluglu.append(redirector + line[4:]) 
                    elif s.name in ["WZ_UL2017"]:
                        chain_WZ.append(redirector + line[4:]) 
                    else:
                        chain.append(redirector + line[4:]) 

In [9]:
%%time
def execute_MC(chain, branchlist_, outFilePath = "./preselectionUL.root", outTreeName = "Events",  nPart = nPartitions, useFlag_BadPFMuonDzFilter = True, LHE = True, label = "distrdf" ):
    if distributed == True:
        #df = RDataFrame("Events", chain, npartitions=nPart, daskclient=client, monitor_label = label)  #when using root version with monitoring features (/cvmfs/images.dodas.infn.it/registry.hub.docker.com/dodasts/root-in-docker:ubuntu22-kernel-v1-monitoring)
        df = RDataFrame("Events", chain, npartitions=nPart, daskclient=client)  #when using standard root versions

    else:
        #df = RDataFrame("Events", chain) #to run on all
        df = RDataFrame("Events", chain[0])  
    
    ### book snapshot ####
    opts = ROOT.RDF.RSnapshotOptions()
    opts.fLazy = True

    if LHE == False:
        branches = branchlist_[4:]
    else:
        branches = branchlist_
    if distributed == True:
        df_lazy = df.Snapshot(outTreeName, outFilePath, branches, opts)
    else:
        df_lazy = df.Snapshot(outTreeName, outFilePath, branches, opts)
    
    events = df.Count()

    return df_lazy

CPU times: user 3 μs, sys: 0 ns, total: 3 μs
Wall time: 4.77 μs


In [10]:
df_sn = execute_MC(chain, branchlist, outFilePath = "./preselectionUL.root", outTreeName = "Events", label = "main") 

/usr/local/lib/root/DistRDF/Backends/Dask/__init__.py:39: FutureWarning: The keyword argument 'daskclient' is not necessary anymore and will be removed in a future release. To provide the distributed.Client object, use 'executor' instead.
  warnings.warn(msg_warn, FutureWarning)


In [11]:
%%time
if distributed == True:
    RunGraphs = ROOT.RDF.Experimental.Distributed.RunGraphs   
else:
    RunGraphs = ROOT.RDF.RunGraphs

proxies = [
    df_sn
]

RunGraphs(proxies)
dfs = [df_.GetValue() for df_ in proxies]

CPU times: user 23.6 s, sys: 2.72 s, total: 26.3 s
Wall time: 1h 44min 11s


Warning in <TStreamerInfo::Build>: ROOT::RDF::RSnapshotOptions: optional<int> has no streamer or dictionary, data member "fBasketSize" will not be saved
Warning in <TStreamerInfo::Build>: _Optional_base<int,true,true>: base class _Optional_base_impl<int,_Optional_base<int,true,true> > has no streamer or dictionary it will not be saved
Warning in <TStreamerInfo::Build>: _Optional_base<int,true,true>: _Optional_payload<int,true,true,true> has no streamer or dictionary, data member "_M_payload" will not be saved
Warning in <TStreamerInfo::Build>: optional<int>: base class _Optional_base<int,true,true> has no streamer or dictionary it will not be saved
Warning in <TStreamerInfo::Build>: optional<int>: base class _Enable_copy_move<true,true,true,true,optional<int> > has no streamer or dictionary it will not be saved
Error in <TStreamerInfo::Build>: _Optional_payload_base<int>, unknown type: _Optional_payload_base<int>::_Storage<int,true> _M_payload

Warning in <TStreamerInfo::Build>: _Optio

In [12]:
print(f"/proc/diskstats post run:\n" + client.run_on_scheduler(print_proc_diskstats))

print(f"Folder size:\n" + client.run_on_scheduler(dir_size))

print(f"Memory info post run:\n" + client.run_on_scheduler(get_memory_info))

/proc/diskstats post run:
 259       0 nvme2n1 70994661 38167099 11940867813 30205243 37980979 748861967 6284883683 5663116 0 6494927 35868360 0 0 0 0 0 0
 259       1 nvme2n1p1 70994563 38167099 11940865141 30205237 37980979 748861967 6284883683 5663116 0 8554891 35868353 0 0 0 0 0 0
 259       2 nvme0n1 71008147 38128980 11943257480 30183824 37947755 748952653 6285343379 6766706 0 7720268 36950531 0 0 0 0 0 0
 259       6 nvme0n1p1 71008049 38128980 11943254808 30183816 37947755 748952653 6285343379 6766706 0 9862847 36950523 0 0 0 0 0 0
 259       3 nvme4n1 71055041 38209743 11934730794 30222403 37891217 748848549 6284058243 5426826 0 6574307 35649229 0 0 0 0 0 0
 259       7 nvme4n1p1 71054943 38209743 11934728122 30222396 37891217 748848549 6284058243 5426826 0 8530916 35649222 0 0 0 0 0 0
 259       4 nvme1n1 22816 66 2760159 5248 5216884 166597 98922932 5916889 0 135205 5922137 0 0 0 0 0 0
 259      12 nvme1n1p1 106 0 2994 12 70 1639 13588 749 0 751 762 0 0 0 0 0 0
 259      13 